# Projet M1 Fouilles de Données, Université Paris 8

## Auteur : KARUNAKARAN Nithushan

https://gitlab.com/fouilles_donnees/Fouilles_Donn-es_Textuelles

In [1]:
import pandas as pd
import numpy as np
import re
import emoji
from nltk.tokenize import TweetTokenizer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
import sys
from unidecode import unidecode
import nltk

In [3]:
pd.set_option('display.max_columns', 10)

In [4]:
from keras.preprocessing.text import Tokenizer

2024-04-13 13:00:35.687182: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
from keras.utils import pad_sequences, to_categorical
from keras.layers import Embedding,Bidirectional,LSTM,Dropout, Dense,Flatten
from keras import Sequential
from keras.models import load_model

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
from gensim.models import KeyedVectors

In [9]:
from sklearn.metrics import accuracy_score

In [10]:
tknzr = TweetTokenizer()
lemmatizer = WordNetLemmatizer()
notstopwords = set(('not', 'no' 'never' 'nor'))
stopwords = set(nltk.corpus.stopwords.words('english')) - notstopwords

In [11]:
emojis = {
    u":)":"😀",
    u":-\]":"😀",
    u":\]":"😀",
    u":-3":"😀",
    u":->":"😀",
    u":3":"😀",
    u"8-\)":"😀",
    u":o\)":"😀",
    u":\}":"😀",
    u":-\}":"🫤",
    u":c\)":"😦",
    u"=\]":"🫤",
    u"=\)":"😀",
    u":)":"😀",
    u":-]":"😀",
    u":]":"😀",
    u"8-)":"😀",
    u":o)":"😮",
    u"(^_^;)":"😳",
    u"(- -);":"😳",
    u"(-_-)zzz":"😳",
    u"(^_-)":"😉",
    u"((+_+))":"😳",
    u"(+o+)":"😀",
    u"^_^":"😳",
    u"(^_^)/":"☺️",
    u"(^<^)":"☺️",
    u"(^^)":"☺️",
    u"(^.^)":"😀",
    u"(^_^.)":"☺️",
    u"(^_^)":"😀",
    u"(^)o(^)":"😀",
    u"(^0^)":"😀",
    u"o_0":"😮",
    u"o.O":"😮",
    u"(o.o)":"😮",
    u"o0":"😮",
    u":D":"😀",
    u"xD":"😀",
    u":(":"😦",
    u":p":"😀",
    u"=p":"😀",
}

In [12]:
def str2emoji(tweet):
    for pos,ej in enumerate(tweet):
        if ej in emojis:
            tweet[pos] = emojis[ej]
            return tweet
    return tweet

In [13]:
def standardization(tweet):
    tweet = re.sub(r"\\u2019","'",tweet)
    tweet = re.sub(r"\\u002c","'",tweet)
    #print("The tweet")
    #print(tweet)
    #print("The str2emoji")
    #print(str2emoji(unidecode(tweet).lower().split()))
    #print("done")
    tweet = ' '.join(str2emoji(unidecode(tweet).lower().split()))
    
    tweet = re.sub(r"(http|https)?:\/\/[a-zA-Z0-9\.-]+\.[a-zA-Z]{2,4}(/\S*)?", " ",tweet)
    tweet = re.sub(r"\'ve", " have",tweet)
    tweet = re.sub(r" can\'t", " cannot",tweet)
    tweet = re.sub(r"n\'t", " not",tweet)
    tweet = re.sub(r"\'re", " are",tweet)
    tweet = re.sub(r"\'d", " would",tweet)
    tweet = re.sub(r"\'ll", " will",tweet)
    tweet = re.sub(r"\'s", "", tweet)
    tweet = re.sub(r"\'n", "", tweet)
    tweet = re.sub(r"\'m", " am",tweet)
    tweet = re.sub(r"@\w+", r' ',tweet)
    tweet = re.sub(r"#\w+", r' ',tweet)
    tweet = re.sub(r" [0-9]+ "," ",tweet)
    tweet = [lemmatizer.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else \
             lemmatizer.lemmatize(i) for i,j in pos_tag(tknzr.tokenize(tweet))]
    tweet = [i for i in tweet if(i not in stopwords) and (i not in punctuation)]
    tweet = ' '.join(tweet)
    return tweet

In [177]:
esl_path = 'Emoji_Sentiment_Data_v1.0.csv'
esl_df = pd.read_csv(esl_path)
print(esl_df.head(20))

   Emoji Unicode codepoint  Occurrences  Position  Negative  Neutral  \
0      😂           0x1f602        14622  0.805101      3614     4163   
1      ❤            0x2764         8050  0.746943       355     1334   
2      ♥            0x2665         7144  0.753806       252     1942   
3      😍           0x1f60d         6359  0.765292       329     1390   
4      😭           0x1f62d         5526  0.803352      2412     1218   
5      😘           0x1f618         3648  0.854480       193      702   
6      😊           0x1f60a         3186  0.813302       189      754   
7      👌           0x1f44c         2925  0.805223       274      728   
8      💕           0x1f495         2400  0.765726        99      683   
9      👏           0x1f44f         2336  0.787130       243      634   
10     😁           0x1f601         2189  0.796151       278      648   
11     ☺            0x263a         2062  0.798634       128      449   
12     ♡            0x2661         1975  0.763695       102     

In [14]:
def data_preprocessing_three(path):
    data = pd.read_csv(path,
                encoding='latin-1')
    data['Tweet'] = data['text'].apply(str).apply(lambda x : standardization(x))
    ret = pd.DataFrame(data, columns=['Tweet', 'sentiment'])
    #return data['Tweet']
    return ret

In [15]:
def data_preprocessing_seven(path):
    data = pd.read_csv(path,sep = '\t', encoding='latin-1')
    data.rename(columns={'Intensity Class': 'sentiment'},inplace=True)
    for val in data["sentiment"]:
        data["sentiment"] = [''.join(re.findall("-\d|\d", item)) for item in data["sentiment"]]

    for i in range(0,len(data["sentiment"])):
        data["sentiment"].iloc[i] = int(data["sentiment"].iloc[i])
    
    data['Tweet'] = data['Tweet'].apply(str).apply(lambda x : standardization(x))
    ret = pd.DataFrame(data, columns=['Tweet', 'sentiment'])
    #return data['Tweet']
    return ret

## Données d'entraînement et test à 3 classes

In [16]:
train = data_preprocessing_three("Datasets/sentiment140/train.csv")

## Données d'entraînement à 7 classes

In [17]:
train_7 = data_preprocessing_seven("Datasets/2018-Valence-oc-En-train.txt")

## Données de test à 7 classes

In [18]:
test_7 = data_preprocessing_seven("Datasets/2018-Valence-oc-En-dev.txt")

In [19]:
all_tweets = pd.concat([train["Tweet"],train_7["Tweet"]])

In [20]:
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(all_tweets)
word_index = tokenizer.index_word

In [21]:
sequence_train = tokenizer.texts_to_sequences(train['Tweet'])

In [22]:
sequence_train_7 = tokenizer.texts_to_sequences(train_7["Tweet"])

In [23]:
MAX_SEQUENCE_LENGTH = 0
for elt in sequence_train:
    if len(elt) > MAX_SEQUENCE_LENGTH:
        MAX_SEQUENCE_LENGTH = len(elt)

In [24]:
data_train = pad_sequences(sequence_train,maxlen=MAX_SEQUENCE_LENGTH)
data_train_7 = pad_sequences(sequence_train_7,maxlen=MAX_SEQUENCE_LENGTH)

In [27]:
indices_train = np.arange(data_train.shape[0])
data_train = data_train[indices_train]

indices_train_7 = np.arange(data_train_7.shape[0])
data_train_7 = data_train_7[indices_train_7]

sentiment= np.array(train['sentiment'])
label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(sentiment)

labels_train = to_categorical(np.asarray(vec),3)


In [28]:
labels_train_7 = to_categorical(np.asarray(train_7['sentiment']),7)

In [30]:
split_idx = int(len(data_train)*0.85)
x_train,x_test = data_train[:split_idx], data_train[split_idx:]
y_train, y_test = labels_train[:split_idx],labels_train[split_idx:]


split_idx_7 = int(len(data_train_7)*0.85)
x_train_7,x_test_7 = data_train_7[:split_idx_7], data_train_7[split_idx_7:]
y_train_7, y_test_7 = labels_train_7[:split_idx_7],labels_train_7[split_idx_7:]

In [31]:
print(data_train_7.shape)
print(x_train_7.shape)
print(x_test_7.shape)
print(y_train_7.shape)
print(y_test_7.shape)

(1181, 36)
(1003, 36)
(178, 36)
(1003, 7)
(178, 7)


In [32]:
x_train.shape

(23358, 36)

In [172]:
print(x_train)
print(x_train_7)

[[    0     0     0 ...     0  1271     3]
 [    0     0     0 ...    22  1222  2026]
 [    0     0     0 ...  1098  2340     1]
 ...
 [    0     0     0 ... 20812    35   791]
 [    0     0     0 ...   173 20813  6055]
 [    0     0     0 ...  1388  1860    47]]
[[   0    0    0 ...  449   86   34]
 [   0    0    0 ...  542 3397 2449]
 [   0    0    0 ... 2187   30 1358]
 ...
 [   0    0    0 ...  363  157   54]
 [   0    0    0 ... 1593 1593  449]
 [   0    0    0 ...  829 1666  135]]


In [33]:

nb_words=len(word_index)+1
oov = []
EMBEDDING_DIM=300
oov.append((np.random.rand(EMBEDDING_DIM)*2.0) -1.0)
oov = oov /np.linalg.norm(oov)
EMBEDDING_FILE="Datasets/GoogleNews-vectors-negative300.bin"
embedding_matrix = np.zeros((nb_words,EMBEDDING_DIM))
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE,binary=True)

In [34]:
vocab = list(word2vec.index_to_key)

In [35]:
for i, word in word_index.items():
    if word in vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
    else:
        embedding_matrix[i]=oov

/tmp/ipykernel_4552/2534983545.py:3: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  embedding_matrix[i] = word2vec.word_vec(word)


In [36]:
embedding_matrix.shape

(23660, 300)

In [37]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights = [embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable = False, name = 'embedding_layer')

## Modèle pour les sentiments à 3 classes

In [149]:
model1 = Sequential()
model1.add(embedding_layer)
model1.add(Bidirectional(LSTM(128, return_sequences=True)))
model1.add(Dropout(0.2))
model1.add(Bidirectional(LSTM(128, return_sequences=True)))
model1.add(Flatten())
model1.add(Dense(3, activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['acc'])
model1.summary()

2024-04-13 15:26:35.823987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-13 15:26:35.825013: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-13 15:26:35.825636: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 36, 300)          7098000   
                                                                 
 bidirectional_11 (Bidirecti  (None, 36, 256)          439296    
 onal)                                                           
                                                                 
 dropout_12 (Dropout)        (None, 36, 256)           0         
                                                                 
 bidirectional_12 (Bidirecti  (None, 36, 256)          394240    
 onal)                                                           
                                                                 
 flatten_4 (Flatten)         (None, 9216)              0         
                                                                 
 dense_16 (Dense)            (None, 3)                

2024-04-13 15:26:36.074131: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-13 15:26:36.075054: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-13 15:26:36.075741: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
early_stopping = EarlyStopping(
                              patience=10,
                              min_delta=0.001,                               
                              monitor="val_loss",
                              restore_best_weights=True
                              )

In [150]:
history=model1.fit(x_train,y_train,validation_data = (x_test,y_test),epochs = 20,batch_size = 50,callbacks=[early_stopping])
model1.save("./model_4.h5")

Epoch 1/20


2024-04-13 15:26:56.254173: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-13 15:26:56.255438: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-13 15:26:56.256429: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

113/468 [======>.......................] - ETA: 2:26 - loss: 0.8865 - acc: 0.5883

KeyboardInterrupt: 

In [159]:
base_3 = load_model("./model_4.h5")

2024-04-13 15:29:06.038416: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-13 15:29:06.039399: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-13 15:29:06.040007: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [51]:
test = data_preprocessing_three("Datasets/sentiment140/test.csv")
test.dropna(inplace = True)
print(test)

                                                  Tweet sentiment
0                                      last session day   neutral
1     shanghai also really exciting precisely skyscr...  positive
2     recession hit veronique branquinho quit compan...  negative
3                                            happy bday  positive
4                                                  like  positive
...                                                 ...       ...
3529                                 im tired sleep try  negative
3530  alone old house thanks net keep alive kick who...  positive
3531  know mean little dog sink depression ... want ...  negative
3532         _sutra next youtube video gonna love video  positive
3533                             omgssh ang cute ng bby  positive

[3534 rows x 2 columns]


In [65]:
sequence_test = tokenizer.texts_to_sequences(test['Tweet'])
data_test = pad_sequences(sequence_test,maxlen=MAX_SEQUENCE_LENGTH)

indices_test = np.arange(data_test.shape[0])
data_test = data_test[indices_test]

In [66]:
print(data_test)

[[   0    0    0 ...   44  935    4]
 [   0    0    0 ... 1499 1739 5668]
 [   0    0    0 ... 1095  597  615]
 ...
 [   0    0    0 ...  196 9705 7109]
 [   0    0    0 ...   65   10  263]
 [   0    0    0 ...  268 2080 3102]]


In [67]:
sentiment_test= np.array(test['sentiment'])
label_encoder_test = LabelEncoder()

In [68]:
vec_test = label_encoder_test.fit_transform(sentiment_test)

labels_test = to_categorical(np.asarray(vec_test),3)

In [69]:
print(vec_test)

[1 2 0 ... 0 2 2]


In [58]:
result_3 = base_3.evaluate(data_test,labels_test,batch_size = 100)

36/36 [==============================] - 3s 72ms/step - loss: 1.8150 - acc: 0.7012


In [72]:
print(data_test[1])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0   161    24  1066 13264  5828     7  1499  1739  5668]


In [103]:
#predict_3 = base_3.predict(data_test).astype("int32")
predict_3 = base_3.predict(data_test)
predict_a = predict_3[5]
print(predict_a[0])
print(predict_a[1])
print(predict_a[2])
#print('{:.20f}'.format(predict_3[1]))
#print(predict_3)
#print(labels_test)
print(labels_test[5])

111/111 [==============================] - 3s 26ms/step
0.0032769404
0.025360508
0.97136265
[0. 0. 1.]


In [160]:
model_7 = Sequential(base_3.layers[:-1]) # avec flatten
#model_7 = Sequential(base_3.layers[:-2]) # sans flatten

2024-04-13 15:29:12.269293: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-13 15:29:12.270540: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-13 15:29:12.271334: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [161]:
for l in model_7.layers:
    l.trainable = False

In [162]:
for l in model_7.layers:
    print(l.name, l.trainable)

embedding_layer False
bidirectional False
dropout False
bidirectional_1 False


In [134]:
model_7.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 36, 300)          7098000   
                                                                 
 bidirectional (Bidirectiona  (None, 36, 128)          186880    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 36, 128)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 36, 128)          98816     
 nal)                                                            
                                                                 
Total params: 7,383,696
Trainable params: 0
Non-trainable params: 7,383,696
_________________________________________________________________


In [135]:
model_7.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 36, 300)          7098000   
                                                                 
 bidirectional (Bidirectiona  (None, 36, 128)          186880    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 36, 128)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 36, 128)          98816     
 nal)                                                            
                                                                 
Total params: 7,383,696
Trainable params: 0
Non-trainable params: 7,383,696
_________________________________________________________________


## Modèle pour les sentiments en 7 classes

In [163]:
model_7.add(Bidirectional(LSTM(128, return_sequences=True)))
model_7.add(Dropout(0.2))
model_7.add(Bidirectional(LSTM(128, return_sequences=True)))
model_7.add(Dropout(0.2))
model_7.add(Flatten())
model_7.add(Dense(7,activation="softmax"))

2024-04-13 15:29:24.539416: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-13 15:29:24.540355: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-13 15:29:24.541095: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [175]:
model_7.compile(optimizer="adam", loss ="categorical_crossentropy", metrics= ["accuracy"])

In [176]:
history = model_7.fit(x_train_7,y_train_7, validation_data = (x_test_7,y_test_7),epochs = 50, batch_size=200)
model_7.save("./model_7_8.h5")

Epoch 1/50


2024-04-13 15:47:18.738167: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-13 15:47:18.739631: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-13 15:47:18.741271: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

6/6 [==============================] - ETA: 0s - loss: 1.9216 - accuracy: 0.2682

2024-04-13 15:47:29.073726: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-13 15:47:29.075816: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-13 15:47:29.077079: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

6/6 [==============================] - 12s 860ms/step - loss: 1.9216 - accuracy: 0.2682 - val_loss: 1.9099 - val_accuracy: 0.2697
Epoch 2/50
6/6 [==============================] - 3s 540ms/step - loss: 1.8689 - accuracy: 0.3689 - val_loss: 1.8879 - val_accuracy: 0.2753
Epoch 3/50
6/6 [==============================] - 3s 450ms/step - loss: 1.8225 - accuracy: 0.3789 - val_loss: 1.8644 - val_accuracy: 0.2753
Epoch 4/50
6/6 [==============================] - 3s 468ms/step - loss: 1.7786 - accuracy: 0.3918 - val_loss: 1.8389 - val_accuracy: 0.2697
Epoch 5/50
6/6 [==============================] - 3s 541ms/step - loss: 1.7386 - accuracy: 0.3848 - val_loss: 1.8176 - val_accuracy: 0.2978
Epoch 6/50
6/6 [==============================] - 4s 574ms/step - loss: 1.6901 - accuracy: 0.3928 - val_loss: 1.8012 - val_accuracy: 0.2809
Epoch 7/50
6/6 [==============================] - 4s 602ms/step - loss: 1.6324 - accuracy: 0.4008 - val_loss: 1.7885 - val_accuracy: 0.3034
Epoch 8/50
6/6 [==============

In [139]:
test_7 = data_preprocessing_seven("Datasets/2018-Valence-oc-En-dev.txt")

In [140]:
sequence_test_7 = tokenizer.texts_to_sequences(test_7["Tweet"])

In [141]:
data_test_7 = pad_sequences(sequence_test_7,maxlen=MAX_SEQUENCE_LENGTH)

In [142]:
indices_test_7 = np.arange(data_test_7.shape[0])
data_test_7 = data_test_7[indices_test_7]

In [143]:
labels_test_7 = to_categorical(np.asarray(test_7['sentiment']),7)

In [144]:
y_pred_7 = model_7.predict(data_test_7)

2024-04-13 15:18:50.524792: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-13 15:18:50.525997: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-13 15:18:50.527080: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

15/15 [==============================] - 3s 54ms/step


In [145]:
y_pred_7 = np.argmax(y_pred_7, axis=1)

In [146]:
labels_test_7 = np.argmax(labels_test_7, axis=1)

## Mapping des valeurs de sentiments de 0 à 6  en -3 à 3

In [147]:
def map_range(val, old_a, old_b, new_c, new_d):
       return int((val-old_a) / (old_b-old_a) * (new_d-new_c) + new_c)

In [148]:
final_pred = y_pred_7
for i in range(0,len(y_pred_7)):
    final_pred[i] = map_range(y_pred_7[i],0,6,-3,3)